# Inversion and Optimisation  <a class="tocSkip"></a>

## Lecture 11b: Overview and Global Methods  <a class="tocSkip"></a>
This last notebook contains an overview of the various gradient-based optimisation techniques we have covered, and provides a brief outlook on some optimisation techniques that do not fall in this category: methods to find global optima in particular also for discrete problems.

<font size="1pt">Some $\LaTeX$ definitions hidden in this cell (double-click to reveal)</font>
$
\newcommand\vec[1]{\mathbf{#1}}
\newcommand\vlam{\boldsymbol{\lambda}}
\newcommand\vmu{\boldsymbol{\mu}}
\newcommand\mat[1]{\underline{\mathbf{#1}}}
\newcommand\R{\mathbb{R}}
\newcommand\todo[1]{\textcolor{red}#1}
$

# Overview Gradient-Based Optimisation Techniques
In all optimisation techniques we have discussed so far, we have assumed that the function $f$ is continuous and differentiable. Searching for local minima of $f$ is then performed by finding the stationary points of $f$, i.e. finding points $\vec x$ where $f'(\vec x)=0$. This shows the connection to the more general problem of root-finding : finding $\vec x$ where $F(\vec x)=0$. This connection is not entirely symmetric: not all root-finding problems can be formulated as an optimisation problem, because not all functions $F:\R^n\to \R^n$ have a "primitive" function (anti-derivative) $f$ such that $F(\vec x)=f'(\vec x)$. Reversely, not all roots of $f'(\vec x)$ are actually local minima or maxima.

<img src="L11b-slides-1.png">

This relation shows that quadratic optimisation problems are performed by solving linear systems. Efficient linear solutions techniques are not just useful for quadratic minimisation however. In particular, in Newton's method we still need to solve a linear system in each iteration. More generally linear solvers are applied everywhere in computational science and engineering. In many applications the linear solvers are the bottle-neck to good and scalable performance.

<img src="L11b-slides-2.png">

The choice between direct solvers and iterative solvers is usually one of scaling. For small problems, direct solvers are fast and robust. Depending on the type of problem, the cost of direct solvers may increase rapidly with  problem size. Krylov subspace methods are the most popular iterative methods, with Conjugate Gradient *the* method of choice for Symmetric Positive Definite matrices. The cost of iterative solvers can typically be related to the conditioning which can be improved with preconditioners. For certain types of problems (in particular related to elliptic PDEs such as Poisson's problem), multigrid methods can be extremely powerful and achieve (near) linear scaling. Another important consideration when choosing a suitable solver is whether the algorithm is easily parallelisable, and scales well with increasing number of processes.

<img src="L11b-slides-3.png">

## Constrained Optimisation (\*optional material)
In lecture 10, we briefly discussed optimisation problems with constraints, which were formulated in the following form:

\begin{align*}
  \text{minimize}\;\; f(\vec x)& \\
  \text{subject to:}\;\; \vec g(\vec x) &= \vec 0 \\
  \vec h(\vec x) &\leq \vec 0
\end{align*}

where $f:\R^n\to\R, \vec g:\R^n\to\R^p, \vec h:\R^n\to\R^q$ are continuously differentiable. The **Karush-Kuhn-Tucker equations**, provide the conditions for finding local minima of the constrained problem:

\begin{align*}
  f'(\vec x^*) + \vlam^T\vec g'(\vec x^*) + \vmu^T\vec h'(\vec x^*) &= \vec 0 \\
  \vec g(\vec x^*) &= \vec 0 \\
  \vec h(\vec x^*) &\leq \vec 0 \\
  \vmu^T &\geq \vec 0 \\
  \vmu^T \vec h(\vec x) &= 0 
\end{align*}

### Sequential Quadratic Programming <a class="tocSkip"></a>
Just like for unconstrained problems, Newton's method can be seen as a sequence of quadratic minimisation problems based on the quadratic approximation

$$
  f_{\text{quad}}(\vec x^{(k)}+\vec p_{\vec x}) = f(\vec x^{(k)}) + f'(\vec x^{(k)})\vec p_{\vec x} + \tfrac 12 \vec p_{\vec x}^T f''(\vec x^{(k)})\vec p_{\vec x}
$$

of $f(\vec x)$ around $\vec x=\vec x^{(k)}$ (see notebook 5 where we discussed the trust region method), *unconstrained problems* can be solved by a sequence of quadratic minimisation problems with linearized constraints:

\begin{align*}
  & \text{minimize } & f_{\text{quad}}(\vec x^{(k)}+\vec p_{\vec x}) &= f(\vec x^{(k)}) + f'(\vec x^{(k)})\vec p_{\vec x} + \tfrac 12 \vec p_{\vec x}^T\nabla_{\vec x}^2 \mathcal{L}(\vec x^{(k)}, \vlam^{(k)}, \vmu^{(k)})\vec p_{\vec x} \\
  & \text{subject to} & \vec g_{\text{lin}}(\vec x^{(k)}+\vec p_{\vec x}) &= \vec g(\vec x^{(k)}) + \vec g'(\vec x^{(k)}) \vec p_{\vec x} = \vec 0 \\
  & & \vec h_{\text{lin}}(\vec x^{(k)}+\vec p_{\vec x}) &= \vec h(\vec x^{(k)}) + \vec h'(\vec x^{(k)}) \vec p_{\vec x} \leq \vec 0,
\end{align*}

where $\mathcal{L}$ is the Lagrangian:

$$
  \mathcal{L}(\vec x^{(k)}, \vlam^{(k)}, \vmu^{(k)}) = f(\vec x) + \vlam^T \vec g(\vec x) + \vmu^T \vec h(\vec x)
$$

See lecture 11 for more details. In python use [SLSQP](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-slsqp.html) available in scipy.

### Interior Point/Penalty Methods <a class="tocSkip"></a>
An alternative approach seeks a solution to the constrained optimisation problem through a Sequence of Unconstrained Minimization Problems (SUMT). The unconstrained problems employ a penalty term that keeps the solution away from the boundary imposed by $h(\vec x)\lt 0$. Interior penalty methods prevent the solution from actually leaving the feasible domain by ramping up the penalty term to infinity already *inside the domain* (usually a $\log$-term). Since depending on the scaling of the penalty term this will always keep the solution at some distance from the feasible boundary, this requires a sequence in which the scaling is subsequently turned down to enable local minima that are very close or at the feasible boundary. A popular implementation of this method is found in [Ipopt](https://projects.coin-or.org/Ipopt) with python binding via [pyipopt](https://code.google.com/archive/p/pyipopt/).

# Global Optimisation (\*optional material)
The gradient based optimisation algorithms we have discussed in this course, only guarantee to find a local minimum. The only way to guarantee that this is the global minimum is if the functional is known to have a unique minimum, e.g. for convex functions.

There are of course cases where we would like to know the global minimum. For optimisation problems with a function of continuous variables, unless we know more about the function it can never be guaranteed that the best solution so far is the global optimum, as there can always be some very local part of the domain that we haven't explored with a lower minimum. In fact if we don't even know whether the function is continuous, the global minimum could be taken in a completely isolated point with surrounding function values that are much higher. In *discrete* optimisation problems we could in theory brute search all possibilities (assuming the discrete parameter space is bounded), but for most practical applications the number of possibilities would simply be too big.

<img src="http://rs.io/wp-content/uploads/2014/08/optimization-space.jpg">
<small>Image source: <a href="https://rs.io/ultimate-guide-simulated-annealing">https://rs.io/ultimate-guide-simulated-annealing/</a></small>

## Simulated Annealing
Simulated Annealing is a widely used technique to find global optima in the presence of large numbers of local optima.  "Annealing" refers to an analogy with a process used in metallurgy of heating following by controlled cooling down of metal to decrease stresses and defects in the material. In simulated annealing the temperature controls the amount of randomness in the iterative steps. When the temperature is high, the process is allowed to take large steps which do not necessarily all result in immediate improvements of the functional. This enables the the algorithm to explore and jump between different "valleys" (basins). As the temperature is cooled down the steps are more restrictive with higher probability for steps that actually reduce the functional. This makes it behave more like a standard descent algorithm. Some randomness is still allowed to avoid being trapped in the wrong basin.

The algorithm consists of three steps

1. Choose a random step $\vec p^{(k)}$ for an update

$$
  \vec x^{(k+1)} = \vec x^{(k)} + \vec p^{(k)}
$$

Depending on the exact implementation the step size may also be controlled by temperature, reducing the step size as the temperature is cooled down

2. Decide whether to accept the new iterate. This is typically based on a criterion of the form

$$
  \exp\left( -\frac{f(\vec x^{(k+1)}) - f(\vec x^{(k+1)})}{T}\right) \gt R(0, 1),
$$

where $R(0,1)$ is a random number between 0  and 1. This means that if the functional $f$ has been reduced, the new step is always accepted. Steps that increase $f$ are sometimes accepted with a chance that get smaller as the temperature decreases.

3. Update the temperature: this is typically an exponential decay with $T\to 0$ after a large number of iterations.

There's quite a few parameters that influence the behaviour of the model, such as initial temperature, the decay rate and the (initial) step size. This method may therefore require a bit of tweaking to get good performance for a specific application.

<img src="hill.gif">
<small>Figure source: <a href="https://en.wikipedia.org/wiki/Simulated_annealing">wikipedia</a></small>

## Basinhopping
Note that the previous algorithm does not require any gradient information at all. This means that within a valley/basin the convergence towards a local miniminum may be slow. If gradient information is available, basinhopping, which combines a probability driven algorithm like simulated annealing with gradient based optimisation within each local basin to quickly find the local minimum. For instance the simulated annealing algorithm sketched in the previous section, can be altered to do a local minimisation after step 1. It is then the new value in the new local minimum that is used to determine whether to accept this as a new iterate or not. Care must be taken that the step size is large enough that the algorithm doesn't continue to keep exploring the same basin.

An [implementation in scipy](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.optimize.basinhopping.html) is available that can be combined with any optimisation technique provided by scipy's `minimize`.

## Particle Swarm Optimisation
Global optimisation methods that do not rely on gradient information often invoke (partly) randomised search directions with frequent evaluations of the function, which therefore needs to be relatively cheap to compute. A natural extension to exploring the search space using a single trajectory, is to have multiple particles/agents explore this space in parallel which may help to ensure all parts of the space are covered. As individual particles discover better directions and solutions for minimising the function however, it may be beneficial to share this information with other particles rather than each particle moving around completely independently. This is the core idea behind **Particle Swarm Optimisation**. In its most basic form each particle moves under its own velocity (momentum) that is updated with random perturbations, but the velocity is also nudged towards the best global (i.e. amongst all particles) solution that has been found so far. There are very many variations and extensions to this basic idea. For instance, to avoid all particles being drawn towards a good local minimum that was found early on, it may be better to share information only with a subset of the particles that are more local (close in distance), or the global swarm is subdivided in "tribes" that share information (regardless of distance). In general terms, these various different versions try to find a balance between exploration, i.e. ensuring that the entire search space is covered sufficiently well, and exploitation of succesful solutions that have been found. These various extensions may also mean that the method relies on a large number of parameters that can have an impact on the performance.

<img src="https://upload.wikimedia.org/wikipedia/commons/e/ec/ParticleSwarmArrowsAnimation.gif"></img>
<small>Image source: [wikipedia](https://upload.wikimedia.org/wikipedia/commons/e/ec/ParticleSwarmArrowsAnimation.gif)</small>

## Genetic Algorithms

Genetic algorithms form a wide class of optimisation algorithms which are inspired by the process of natural selection in biological evolution. The optimisation problems are often of a combinatorial nature, where each solution is described by a sequence of discrete values. Some typical examples are:

* a routing problem such as the travelling salesman problem
* networking problems where a large set of "nodes" needs to connected via pipes or lines in some optimal way
* complex molecular structures where substructures can be linked up in many different ways
* the training of neural networks

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*BYDJpa6M2rzWNSurvspf8Q.png"></img>
<small>Figure source: [towardsdatascience.com](https://towardsdatascience.com/introduction-to-genetic-algorithms-including-example-code-e396e98d8bf3), which contains a nice tutotrial with example code
</small>

There is a wide variety of approaches, but some common ingredients include:
- a **fitness function** (the functional) that determines the likelihood of survival
- random changes are introduced via **mutation**: randomly selected entries of the sequence are altered
- **crossover** is the process where two parent solutions combine parts of their sequence to offspring: a new candidate solution that is added to the "gene pool"

Note that the key difference with other randomized search algorithms is the fact that there is a pool of candidates that can share information about **partial** good solutions via crossover. The information sharing is thus much more sophisticated than simply nudging other agents in a good direction as we saw in particle swarm methods.

The candidate selection may be organised via generations. Where each generation only a set amount of the fittest individuals are selected to be included in the next generation. The likelihood of two parents being combined to create offspring for the next generation may also depend on their fitness. In a lot of applications it is quite common that a lot of candidates, particularly in the early generations, are completely disfunctional (e.g. broken networks). It is important that the pool of candidates is large enough for good candidates to emerge.

Genetics algorithms offers a very powerful and versatile methodology that has been succesfully applied in many areas. It can however take considerable time to find the right parameters, and select those techniques that combine to an effective optimisation strategy for a specific problem area.